# Ungraded Lab: Practice with the Keras Functional API

This lab will demonstrate how to build models with the Functional syntax. You'll build one using the Sequential API and see how you can do the same with the Functional API. Both will arrive at the same architecture and you can train and evaluate it as usual.

## Imports

In [ ]:
import pydot
import tensorflow as tf
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

## Sequential API

Here is how we use the `Sequential()` class to build a model.

In [ ]:
def build_model_with_sequential():
    """
    instantiate a Sequential class and linearly stack the layers
    of your model
    """
    seq_model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation=tf.nn.relu),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        ]
    )
    return seq_model

## Functional API

And here is how you build the same model above with the functional syntax.

```python
tf.keras.layers.Dense(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
```

In [ ]:
def build_model_with_functional_api():
    input_layer = tf.keras.layers.Input(shape=(28, 28), name='input')
    flattened = tf.keras.layers.Flatten()(input_layer)
    first_layer = tf.keras.layers.Dense(128, activation=tf.nn.relu, name='first_layer')(flattened)
    output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='output_layer')(first_layer)
    # Once the dependency graph is ready, let's define the model
    func_model = tf.keras.Model(input_layer, output_layer, name='my_model')
    return func_model

## Build the model and visualize the model graph

You can choose how to build your model below. Just uncomment which function you'd like to use. You'll notice that the plot will look the same.

In [ ]:
model = build_model_with_functional_api()
#model = build_model_with_sequential()

# Plot model graph
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')

In [ ]:
model.summary()

In [ ]:
model.name

In [ ]:
model.get_layer('first_layer').weights[0]

In [ ]:
n, bins, patches = plt.hist(
    model.get_layer('first_layer').weights[0].numpy().flatten(),
    50,
    facecolor='b',
    alpha=0.75
)

In [ ]:
print(model.layers[2].weights[0].shape, '\n', model.layers[3].weights[0].shape)

## Training the model

Regardless if you built it with the Sequential or Functional API, you'll follow the same steps when training and evaluating your model.

In [ ]:
# prepare fashion mnist dataset
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images / 255.0
test_images = test_images / 255.0

In [ ]:
training_images.shape, test_images.shape

In [ ]:
training_labels, test_labels

### Loss

In [ ]:
y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
loss.numpy()

In [ ]:
y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.05, 0.95, 0], [0.1, 0.1, 0.8]]
loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
loss.numpy()

In [ ]:
y_true = [[0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
y_pred = [[0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
loss.numpy()

In [ ]:
y_true = [1, 2] # -> [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]] # -> Model's outputs is supposed to be this way
loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
loss.numpy()

### Metrics

In [ ]:
# y_true = one_hot(y_true) = [[0, 1, 0], [0, 0, 1]]
# logits = log(y_pred)
# softmax = exp(logits) / sum(exp(logits), axis=-1)
# softmax = [[0.05, 0.95, EPSILON], [0.1, 0.8, 0.1]]
# xent = -sum(y * log(softmax), 1)
# log(softmax) = [[-2.9957, -0.0513, -16.1181],
#                [-2.3026, -0.2231, -2.3026]]
# y_true * log(softmax) = [[0, -0.0513, 0], [0, 0, -2.3026]]
# xent = [0.0513, 2.3026]
# Reduced xent = (0.0513 + 2.3026) / 2
m = tf.keras.metrics.SparseCategoricalCrossentropy()
m.update_state([1, 2],
               [[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
m.result().numpy()

In [ ]:
m.reset_state()
m.update_state([1, 2],
               [[0.05, 0.95, 0], [0.1, 0.8, 0.1]],
               sample_weight=tf.constant([0.3, 0.7]))
m.result().numpy()

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(),
    loss=tf.keras.metrics.sparse_categorical_crossentropy,
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(),
        tf.keras.metrics.SparseCategoricalCrossentropy()
    ]
)

In [ ]:
model.fit(
    training_images,
    training_labels,
    epochs=5
)

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
model.metrics_names